In [1]:
import tensorflow as tf
import functions
import numpy as np
import csv

# 세션 시작

In [2]:
#그래프로 연결되는 형식
const = tf.constant(10)

with tf.Session() as sess:
    ten = sess.run(const)
    print(ten)

10


# 1. Esimator 사용하기

http://bcho.tistory.com/1196

In [6]:
def input_fn():
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]
    features = tf.constant([up if i % 2 == 0 else down for i in range(1000)], tf.float32)
    label = tf.constant([[1] if i%2 == 0 else [0] for i in range(1000)], tf.float32)
    return features, label #기본적으로 데이터에 대한 2가지만 돌려주기

def model(features, labels, mode):
    layer1 = tf.layers.dense(features, 10)
    layer2 = tf.layers.dense(features, 10)
    layer3 = tf.layers.dense(features, 10)
    layer4 = tf.layers.dense(features, 10)
    out = tf.layers.dense(layer4, 1)
    
    global_step = tf.train.get_global_step() #min. method가 하나씩 올려 줌
    loss = tf.losses.sigmoid_cross_entropy(labels, out)
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss, global_step)
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op) #모델에서는 EstimatorSpec으로 접근 해 주어야함

In [7]:
est = tf.estimator.Estimator(model)
est.train(input_fn, steps=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp1b7ovk5f', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp1b7ovk5f/model.ckpt.
INFO:tensorflow:loss = 0.00493742, step = 1
INFO:tensorflow:Saving checkpoints for 5 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp1b7ovk5f/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00413364.


# 2. 간단하게 Dataset Class로 구현

In [7]:
import os

os.getcwd()

'/Users/naver/nlp_ryan/Tensorflow_basic'

In [12]:
# csv모듈로 데이터 생성
def gen_dataset():
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]
    
    with open("./test.csv", 'w') as f:
        writer = csv.writer(f, delimiter=",")
        for i in range(5):
            writer.writerow([1]+up)
            writer.writerow([0]+down)

In [14]:
# TF를 활용한 데이터셋 로드하기

dataset = tf.contrib.data.TextLineDataset('./test.csv').shuffle(2000).batch(5)
#다수의 파일일 경우 ['1.csv', '2.csv'. .....] 이렇게 리스트 형태로 넣어도 가능
#shuffle과 batch로..

itr = dataset.make_one_shot_iterator() #queue runners동작에서 iterator는 그 queue안에다가 dequeue를 넣어주는 역할
batch = itr.get_next() #batch가 실행되면 다음 batch를 얻어옴

data = tf.decode_csv(batch, [[0]] * 11)
train = tf.stack(data[1:], axis=1) #추가 참조
label = tf.expand_dims(data[0], axis =1)

#머신러닝 모델 연산을 위해 float type casting
train = tf.cast(train, tf.float32)
label = tf.cast(label, tf.float32)

# 3. Evaluate 활용

In [15]:
tf.logging.set_verbosity(tf.logging.INFO) #자체적으로 loss, step acc에 대해서는 로깅을함

def input_fn():
    # TF를 활용한 데이터셋 로드하기

    dataset = tf.contrib.data.TextLineDataset('./test.csv').shuffle(2000).batch(5)
    #다수의 파일일 경우 ['1.csv', '2.csv'. .....] 이렇게 리스트 형태로 넣어도 가능
    #shuffle과 batch로..

    itr = dataset.make_one_shot_iterator() #queue runners동작에서 iterator는 그 queue안에다가 dequeue를 넣어주는 역할
    batch = itr.get_next() #batch가 실행되면 다음 batch를 얻어옴

    data = tf.decode_csv(batch, [[0]] * 11)
    train = tf.stack(data[1:], axis=1) #추가 참조
    label = tf.expand_dims(data[0], axis =1)
    
    #머신러닝 모델 연산을 위해 float type casting
    train = tf.cast(train, tf.float32)
    label = tf.cast(label, tf.float32)
    
    return train, label

def model_fn(features, labels, mode):
    
    layer1 = tf.layers.dense(features, 10)
    layer2 = tf.layers.dense(layer1, 10)
    layer3 = tf.layers.dense(layer2, 10)
    layer4 = tf.layers.dense(layer3, 10)
    layer5 = tf.layers.dense(layer4, 10)
    out = tf.layers.dense(layer5, 1)
    
    global_step = tf.train.get_global_step() #min. method가 하나씩 올려 줌
    loss = tf.losses.sigmoid_cross_entropy(labels, out)
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss, global_step)
    
    acc = tf.metrics.accuracy(labels, tf.round(tf.nn.sigmoid(out)))
    eval_metric_ops = {"acc": acc} #acc, recall, precision을 좀 더 편하게 제공 단지, key와 value를 dict형태로 넣어주어야함
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        train_op=train_op,
        loss = loss,
        eval_metric_ops = eval_metric_ops
        ) #모델에서는 EstimatorSpec으로 접근 해 주어야함

In [16]:
est = tf.estimator.Estimator(model_fn)

for epoch in range(10): #for문이 epoch기능
    est.train(input_fn)
    est.evaluate(input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp5lv5jfyz', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp5lv5jfyz/model.ckpt.
INFO:tensorflow:loss = 1.92598, step = 1
INFO:tensorflow:Saving checkpoints for 2 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp5lv5jfyz/model.ckpt.
INFO:tensorflow:Loss for final step: 0.060384.
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dty

INFO:tensorflow:Saving dict for global step 14: acc = 1.0, global_step = 14, loss = 0.00833884
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp5lv5jfyz/model.ckpt-14
INFO:tensorflow:Saving checkpoints for 15 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp5lv5jfyz/model.ckpt.
INFO:tensorflow:loss = 0.00803016, step = 15
INFO:tensorflow:Saving checkpoints for 16 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp5lv5jfyz/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00819217.
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-09-12-13:11:13
INFO:tensorflow:Restoring paramet

# 실제로 예측해보기

In [17]:
tf.logging.set_verbosity(tf.logging.INFO)


def input_fn():
    """ input function for estimator """
    dataset = tf.contrib.data.TextLineDataset("./test.csv").shuffle(2000).batch(5)
    itr = dataset.make_one_shot_iterator()
    
    batch = itr.get_next()
    
    data = tf.decode_csv(batch, [[0]]*11)
    train = tf.stack(data[1:], axis=1)
    label = tf.expand_dims(data[0], axis=1)
    
    train = tf.cast(train, tf.float32)
    label = tf.cast(label, tf.float32)
    
    return {"x": train}, label


def model_fn(features, labels, mode):
    """ model function for estimator """
    train_op = None
    loss = None
    eval_metric_ops = None
    
    layer1 = tf.layers.dense(features["x"], 10)
    layer2 = tf.layers.dense(layer1, 10)
    layer3 = tf.layers.dense(layer2, 10)
    layer4 = tf.layers.dense(layer3, 10)
    layer5 = tf.layers.dense(layer4, 10)
    out = tf.layers.dense(layer5, 1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        #train, evaluate, predict 값을 가져 갈 수 있음
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions={
                "prob":tf.nn.sigmoid(out)})
    else:
        global_step = tf.train.get_global_step()
        loss = tf.losses.sigmoid_cross_entropy(labels, out)
        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
        
        accuracy = tf.metrics.accuracy(labels, tf.round(tf.nn.sigmoid(out)))
        eval_metric_ops = {"acc": accuracy}
        
        return tf.estimator.EstimatorSpec(
            mode=mode,
            train_op=train_op,
            loss=loss,
            eval_metric_ops=eval_metric_ops)
    
est = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpvr85tv3i', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [18]:
for epoch in range(10):
    est.train(input_fn)
    est.evaluate(input_fn)
    
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array([list(range(10))], np.float32)},
    num_epochs=1,
    shuffle=False)

predictions = est.predict(pred_input_fn)
for i in predictions:
    print(i["prob"])

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpvr85tv3i/model.ckpt.
INFO:tensorflow:loss = 1.92598, step = 1
INFO:tensorflow:Saving checkpoints for 2 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpvr85tv3i/model.ckpt.
INFO:tensorflow:Loss for final step: 0.060384.
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-09-12-13:11:23
INFO:tensorflow:Restoring parameters from /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpvr85tv3i/model.ckpt-2
INFO:tensorflow:Finished evaluation at 2017-09-12-13:11:23
INFO:tensorflow:Saving dict for global step 2: acc = 1.0, global_step = 2, los

INFO:tensorflow:loss = 0.00803016, step = 15
INFO:tensorflow:Saving checkpoints for 16 into /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpvr85tv3i/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00819217.
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-09-12-13:11:47
INFO:tensorflow:Restoring parameters from /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpvr85tv3i/model.ckpt-16
INFO:tensorflow:Finished evaluation at 2017-09-12-13:11:48
INFO:tensorflow:Saving dict for global step 16: acc = 1.0, global_step = 16, loss = 0.00743878
INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmp